# Test plane-based registration
Test different plane-based registration methods

In [ ]:
%matplotlib widget
import numpy as np
import os
import time
import plotly.graph_objects as go

import planeslam.io as io
from planeslam.general import plot_3D_setup, color_legend
from planeslam.scan import pc_to_scan
from planeslam.general import NED_to_ENU

os.environ['KMP_DUPLICATE_LIB_OK']='True'

%load_ext autoreload
%autoreload 2

In [ ]:
np.set_printoptions(suppress=True)

### Load AirSim data

In [ ]:
# Read in point cloud data
binpath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_20_samples_1', 'lidar', 'Drone0')
PC_data = io.read_lidar_bin(binpath)

# Read in ground-truth poses (in drone local frame)
posepath = os.path.join(os.getcwd(), '..', '..', 'data', 'airsim', 'blocks_20_samples_1', 'poses', 'Drone0')
drone_positions, drone_orientations = io.read_poses(posepath)

In [ ]:
# Convert to ENU
num_scans = len(PC_data)

for i in range(num_scans):
    PC_data[i] = NED_to_ENU(PC_data[i])

drone_positions = NED_to_ENU(drone_positions)
drone_orientations = NED_to_ENU(drone_orientations)

In [ ]:
# Plot ground-truth trajectory
gt_traj_trace = go.Scatter3d(x=drone_positions[:,0], y=drone_positions[:,1], z=drone_positions[:,2])
fig = go.Figure(data=gt_traj_trace)
fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# Extract scans
num_scans = len(PC_data)
scans = num_scans * [None]
scans_transformed = num_scans * [None]
for i in range(num_scans):
    scans[i] = pc_to_scan(PC_data[i])

### Decoupled

In [ ]:
from planeslam.registration import decoupled_register

abs_traj_transformations = np.zeros((num_scans-1,4,4))
T_abs = np.eye(4)
T_abs[:3,3] = drone_positions[0]
avg_runtime = 0

for i in range(len(scans)-1):
    start_time = time.time()
    R_hat, t_hat = decoupled_register(scans[i+1], scans[i])
    avg_runtime += time.time() - start_time
    T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.hstack((np.zeros(3), 1))))
    T_abs = T_hat @ T_abs
    abs_traj_transformations[i,:,:] = T_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

traj_est = abs_traj_transformations[:,:3,3]

In [ ]:
# Plot trajectories
est_traj_trace = go.Scatter3d(x=traj_est[:,0], y=traj_est[:,1], z=traj_est[:,2])
fig = go.Figure(data=[gt_traj_trace, est_traj_trace])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
traj_est

In [ ]:
# RMSE error
np.mean(np.linalg.norm(traj_est - drone_positions[1:], axis=1))

Decoupled Gauss Newton

In [ ]:
from planeslam.registration import decoupled_GN_register

abs_traj_transformations = np.zeros((num_scans-1,4,4))
T_abs = np.eye(4)
T_abs[:3,3] = drone_positions[0]
avg_runtime = 0

for i in range(len(scans)-1):
    start_time = time.time()
    R_hat, t_hat = decoupled_GN_register(scans[i], scans[i+1])
    avg_runtime += time.time() - start_time
    T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.hstack((np.zeros(3), 1))))
    T_abs = T_hat @ T_abs
    abs_traj_transformations[i,:,:] = T_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

traj_est = abs_traj_transformations[:,:3,3]

In [ ]:
# Plot trajectories
est_traj_trace = go.Scatter3d(x=traj_est[:,0], y=traj_est[:,1], z=traj_est[:,2])
fig = go.Figure(data=[gt_traj_trace, est_traj_trace])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
# RMSE error
np.mean(np.linalg.norm(traj_est - drone_positions[1:], axis=1))

In [ ]:
R_hat, t_hat = decoupled_GN_register(scans[11], scans[12])

In [ ]:
R_hat

In [ ]:
t_hat

Merging 

In [ ]:
# Transform scans
for i in range(num_scans-1):
    R = abs_traj_transformations[i,:3,:3]
    t = abs_traj_transformations[i,:3,3]
    scans[i+1].transform(R, t)

In [ ]:
# Plot scans
fig = go.Figure()

for i, scan in enumerate(scans):
    for t in scan.plot_trace():
        fig.add_trace(t)

fig.update_layout(width=1000, height=600, scene=dict(aspectmode='data'))
fig.show()

In [ ]:
merged = scans[0]

for s in scans[1:]:
    merged = merged.merge(s)
    #merged.fuse_edges()

In [ ]:
# Plot merge
fig = go.Figure(data=merged.plot_trace())
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()

Test with velodyne

In [ ]:
pcpath = os.path.join(os.getcwd(),'..', '..', 'data', 'velodyne', '8_5_2022', 'flightroom', 'test')

In [ ]:
PCs = []
for i in range(200, 400, 5):  # ignore first 200 and last 200 frames
    filename = pcpath+'/pc_'+str(i)+'.npy'
    PC = np.load(filename)
    PCs.append(PC)

In [ ]:
# Extract scans
num_scans = len(PCs)
scans = num_scans * [None]
scans_transformed = num_scans * [None]
for i in range(num_scans):
    scans[i] = pc_to_scan(PCs[i], ds_rate=5)

In [ ]:
from planeslam.registration import decoupled_GN_register

abs_traj_transformations = np.zeros((num_scans-1,4,4))
T_abs = np.eye(4)
avg_runtime = 0

for i in range(len(scans)-1):
    start_time = time.time()
    R_hat, t_hat = decoupled_GN_register(scans[i], scans[i+1])
    avg_runtime += time.time() - start_time
    T_hat = np.vstack((np.hstack((R_hat, t_hat)), np.hstack((np.zeros(3), 1))))
    T_abs = T_hat @ T_abs
    abs_traj_transformations[i,:,:] = T_abs

avg_runtime /= len(scans)-1
print("average registration time: ", avg_runtime)

traj_est = abs_traj_transformations[:,:3,3]

In [ ]:
# Plot trajectories
est_traj_trace = go.Scatter3d(x=traj_est[:,0], y=traj_est[:,1], z=traj_est[:,2])
fig = go.Figure(data=[est_traj_trace])
fig.update_layout(width=1500, height=900, scene=dict(aspectmode='data'))
fig.show()